In [1]:
import pandas as pd
from datetime import datetime
from osgeo import gdal
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score

#Skyler Added:
from dateutil import parser
from keras.layers import Flatten, Dense

In [2]:
train_labels = pd.read_csv("train_labels.csv")
grid_metadata = pd.read_csv("grid_metadata.csv")
satellite_metadata = pd.read_csv("satellite_metadata.csv")
satellite_metadata['Date'] =  pd.to_datetime(satellite_metadata['time_end'], format='%Y-%m-%d')

In [3]:
####################
# REMOVE THIS LINE #
####################
train_labels = train_labels.sample(1000, random_state=42)
#print(train_labels)

In [4]:
def get_grid_data(metadata, grid_id):
    return metadata[metadata["grid_id"] == grid_id]

In [5]:
def fetch_satellite_meta(metadata, datetime, location, datatype, split):
    if location == "Delhi":
        location = "dl"
    elif location == "Taipei":
        location = "tpe"
    else:
        location = "la"
        
    metadata = metadata[metadata['location'] == location]
    metadata = metadata[metadata['product'] == datatype]
    metadata = metadata[metadata['split'] == split]
    dateobject = parser.parse(datetime)
    return metadata.loc[(metadata['Date'].dt.month == dateobject.month) & 
                        (metadata['Date'].dt.day == dateobject.day) &
                        (metadata['Date'].dt.year <= dateobject.year)]

In [6]:
# Opens the HDF file
def load_data(FILEPATH):
    print(FILEPATH)
    ds = gdal.Open(FILEPATH)
    return ds

def fetch_subset(granule_id):
    #ds = load_data("dataset/" + granule_id)
    #Line Below Changed
    ds = load_data("C:/Users/Skyler/" + granule_id)
    
    ds.GetSubDatasets()[0]
    raster = gdal.Open(ds.GetSubDatasets()[8][0]) #grid5km:cosSZA features only
    band = raster.GetRasterBand(1)
    band_arr = band.ReadAsArray()
    return band_arr

In [7]:
def fetch_training_features(grid_id, datetime, split):
    temp = get_grid_data(grid_metadata, grid_id)
    sat_met = fetch_satellite_meta(satellite_metadata, 
                               datetime, 
                               temp.iloc[0]['location'], 
                               "maiac", 
                               split)
    counter = 0
    features = None
    for i in range(len(sat_met)):
        counter+=1
        #Lines below added
        split = str(sat_met.iloc[i]['split'])
        product = str(sat_met.iloc[i]['product'])
        year = str(sat_met.iloc[i]['Date'].year)
        granule_id = str(sat_met.iloc[i]['granule_id'])
        path_str = split + "/" + product + "/" + year + "/" + granule_id
        
        subset = fetch_subset(path_str)
        if features is None:
            features = subset
        else:
            features+=subset
    return features/counter

In [8]:
def generate_features(train_labels, split):
    labels = []
    features = []
    for i in range(len(train_labels)):
        feature = fetch_training_features(train_labels.iloc[i]['grid_id'], train_labels.iloc[i]['datetime'], split)
        features.append(np.array(feature).reshape(-1))
        if split == "train":
            labels.append(train_labels.iloc[i]['value'])
    return np.array(features), np.array(labels)

In [9]:
features, labels = generate_features(train_labels, "train")

C:/Users/Skyler/train/maiac/2018/20180514T070000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190514T063000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200514T064500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180604T054500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190604T065000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200604T070000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180604T202500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20181225T191500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20191225T202000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20181018T063000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191018T060000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20201018T061000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180218T032500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2018/20180218T032500_maiac_tpe_1.hdf
C:/Users/Skyler/train/maiac/2019/20190218T025500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2019/2019

C:/Users/Skyler/train/maiac/2018/20180718T191500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190718T202000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20200718T203000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180911T192000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180702T191500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190702T202000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180611T055000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190611T065500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200611T070500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180710T065500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190710T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200710T063500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181228T063500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191228T060500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181113T201500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/2019111

C:/Users/Skyler/train/maiac/2019/20190131T030500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2019/20190131T030500_maiac_tpe_1.hdf
C:/Users/Skyler/train/maiac/2018/20180424T203000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190424T200000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20200424T201500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180218T195000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190218T192000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20200218T202500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180421T065500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190421T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200421T063500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181119T031500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2018/20181119T031500_maiac_tpe_1.hdf
C:/Users/Skyler/train/maiac/2019/20191119T024000_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2019/20191119T042000_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2020/2

C:/Users/Skyler/train/maiac/2019/20190527T191000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20181107T191500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20191107T202000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20201107T203000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180625T191000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20181011T031000_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2018/20181011T031000_maiac_tpe_1.hdf
C:/Users/Skyler/train/maiac/2019/20191011T023500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2019/20191011T041500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2018/20180716T062000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190716T055000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200716T060000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180506T192000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190506T202500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20200506T203500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/201

C:/Users/Skyler/train/maiac/2018/20180707T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190707T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200707T060500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180324T031500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2018/20180324T031500_maiac_tpe_1.hdf
C:/Users/Skyler/train/maiac/2019/20190324T024000_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2019/20190324T042000_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2020/20200324T025500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2020/20200324T025500_maiac_tpe_1.hdf
C:/Users/Skyler/train/maiac/2018/20180831T063000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190831T060000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181007T065000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191007T062000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181008T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180316T202500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/2

...

C:/Users/Skyler/train/maiac/2019/20190912T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180906T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190906T070000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181229T202500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180612T063000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190612T060000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200612T061000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190108T062000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200108T054500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180809T071000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181022T191500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20191022T202000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20201022T203000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180725T192000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190725T202500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/2018032

C:/Users/Skyler/train/maiac/2018/20180731T202000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180723T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190723T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181018T063000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191018T060000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180618T204000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190618T201000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20181101T064500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191101T061000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20201101T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181130T061500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191130T054000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20201130T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190127T195500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20200127T192500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/2018112

C:/Users/Skyler/train/maiac/2019/20191107T071500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20201107T055000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181120T202000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20191120T195000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20201120T200000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180714T063000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190714T060000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200714T061000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181123T060500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191123T071500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20201123T055000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180723T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190723T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181130T061500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191130T054000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/2020113

C:/Users/Skyler/train/maiac/2019/20190620T065000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200620T070000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180427T062000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190427T055000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181013T061500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20191013T054000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20201013T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180821T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180621T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190621T055500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200621T060500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181021T201000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180915T203500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190915T200000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2020/20200915T201500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/2018120

C:/Users/Skyler/train/maiac/2019/20190104T032500_maiac_tpe_1.hdf
C:/Users/Skyler/train/maiac/2018/20180928T065500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190928T062500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2020/20200928T063500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180804T065000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190804T062000_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20180226T203500_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2019/20190226T201000_maiac_la_0.hdf
C:/Users/Skyler/train/maiac/2018/20180816T071500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20190816T064500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2018/20181205T031500_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2018/20181205T031500_maiac_tpe_1.hdf
C:/Users/Skyler/train/maiac/2019/20191205T024000_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2019/20191205T042000_maiac_tpe_0.hdf
C:/Users/Skyler/train/maiac/2018/20181022T060500_maiac_dl_0.hdf
C:/Users/Skyler/train/maiac/2019/20

In [10]:
def baseline_model():
    model = Sequential()
    #model.add(Dense(1, input_dim=10, activation='relu'))
    #model.add(Dense(1))
    
    model.add(Flatten())
    model.add(Dense(128, input_dim=10, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)
results = cross_val_score(estimator, features, labels)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

C:\Users\Skyler\AppData\Local\Temp/ipykernel_22472/4120225059.py:14: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)


Baseline: -9229.06 (1392.68) MSE


###### 